# OpenData REST API

## Libraries and Settings

In [1]:
# Libraries
import os
import requests
import json
import pprint

import pandas as pd


In [2]:
# Package list of the Swiss open data portal
baseUrl = 'https://opendata.swiss/api/3/action/package_show'
id = '?id=fc0430b9-4b27-4ea5-a7fe-5cb1cf6eb142'
callUrl = baseUrl + id

# Make the HTTP request
response = requests.get(callUrl)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK
result = response_dict['result']         # extract all the packages from the response
pprint.pprint(result)                    # pretty print the list to the screen

{'accrual_periodicity': 'http://publications.europa.eu/resource/authority/frequency/IRREG',
 'author': None,
 'author_email': None,
 'contact_points': [{'email': 'opendata@zuerich.ch',
                     'name': 'Open Data Zürich'}],
 'creator_user_id': 'af084126-f711-4016-a585-70354e997796',
 'description': {'de': 'Der Datensatz enthält die Resultate der '
                       'Mietpreiserhebung 2022. Enthalten sind die geschätzten '
                       'Mietpreisbandbreiten für vier Raumstufen (Ganze Stadt, '
                       'Stadtkreise, Stadtquartiere und Quartiergruppen). '
                       'Siehe dazu unten unter «Bemerkungen» die weiteren '
                       'Erläuterungen.\n'
                       '\n'
                       'Erleben Sie die Daten auch auf dem interaktiven '
                       '[MPE-Tool](https://www.stadt-zuerich.ch/content/prd/de/index/statistik/publikationen-angebote/datenbanken-anwendungen/mietpreiserhebung.html) '
            

In [3]:
# Print the data format
data_format = result['resources'][0]['format']
print('Data format:  ' + data_format)

Data format:  CSV


In [4]:
# Get the url for the data from the dictionary
data_url = result['resources'][0]['url']
print('Data url:     ' + data_url)

Data url:     https://data.stadt-zuerich.ch/dataset/bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161/download/BAU516OD5161.csv


In [5]:

df = pd.read_csv(data_url)
# drop unnecessary cols
df.drop(df.columns[0:2], axis=1, inplace=True)
df.drop('ZimmerLang', axis=1, inplace=True)
df.drop(df.columns[12:29], axis=1, inplace=True) #inplace=True verändert Daten direkt auf dataframe, asonsten müsste man df = df.drop() machen

In [6]:
# rename cols
df.rename(columns={'RaumeinheitLang': 'Raumeinheit', 'GliederungLang': 'Gliederung', 'GliederungLang': 'Gliederung', 'Domain':'Grundgesamtheit', 'ZimmerSort': 'Zimmer'}, inplace=True)

In [7]:
# only use brutto prices
df = df[df.PreisartLang != 'Netto']
# only use flats, not square meter entries
df = df[df.EinheitLang == 'Wohnung']

In [8]:
# Drop Preisart cols
df.drop('PreisartLang', axis=1, inplace=True)
df.drop('PreisartSort', axis=1, inplace=True)

In [9]:
# Values in column "Raumeinheit"
df['Raumeinheit'].unique()

array(['Ganze Stadt', 'Stadtkreise', 'Quartiergruppen', 'Quartiere'],
      dtype=object)

In [10]:
# Values in column "Gliederung"
df['Gliederung'].unique()

array(['Ganze Stadt', 'Neubau bis 2 Jahre', 'Neubezug bis 2 Jahre',
       'Bestand Mietverträge 2–10 Jahre',
       'Bestand Mietverträge 11-20 Jahre',
       'Bestand Mietverträge über 20 Jahre', 'Kreis 1', 'Kreis 2',
       'Kreis 3', 'Kreis 4', 'Kreis 5', 'Kreis 6', 'Kreis 7', 'Kreis 8',
       'Kreis 9', 'Kreis 10', 'Kreis 11', 'Kreis 12', 'Sihlfeld',
       'Alt-Wiedikon und Friesenberg', 'Hard', 'Werd und Langstrasse',
       'Unterstrass', 'Oberstrass', 'Fluntern und Hottingen',
       'Hirslanden und Witikon', 'Albisrieden', 'Altstetten', 'Höngg',
       'Wipkingen', 'Affoltern', 'Oerlikon', 'Seebach', 'Hirzenbach',
       'Saatlen und Schwamendingen-Mitte', 'Rathaus', 'Hochschulen',
       'Lindenhof', 'City', 'Wollishofen', 'Leimbach', 'Enge',
       'Alt-Wiedikon', 'Friesenberg', 'Werd', 'Langstrasse',
       'Gewerbeschule', 'Escher Wyss', 'Fluntern', 'Hottingen',
       'Hirslanden', 'Witikon', 'Seefeld', 'Mühlebach', 'Weinegg',
       'Saatlen', 'Schwamendingen-Mitte'], 

In [11]:
# Values in column "GemeinnuetzigLang"
df['GemeinnuetzigLang'].unique()

array(['Alle Wohnungen', 'Gemeinnützig', 'Nicht gemeinnützig'],
      dtype=object)

In [12]:
# Values in column "EinheitLang"
df['EinheitLang'].unique()

array(['Wohnung'], dtype=object)

In [13]:
df.head()

,RaumeinheitSort,Raumeinheit,GliederungSort,Gliederung,Zimmer,GemeinnuetzigSort,GemeinnuetzigLang,EinheitSort,EinheitLang,mean,Grundgesamtheit,Sample1,Sample2
108,1,Ganze Stadt,0,Ganze Stadt,2,0,Alle Wohnungen,1,Wohnung,1584.0,46692,13545,1742
109,1,Ganze Stadt,1,Neubau bis 2 Jahre,2,0,Alle Wohnungen,1,Wohnung,2011.0,1498,488,56
110,1,Ganze Stadt,2,Neubezug bis 2 Jahre,2,0,Alle Wohnungen,1,Wohnung,1777.0,13182,3081,488
111,1,Ganze Stadt,3,Bestand Mietverträge 2–10 Jahre,2,0,Alle Wohnungen,1,Wohnung,1609.0,20055,5922,764
112,1,Ganze Stadt,4,Bestand Mietverträge 11-20 Jahre,2,0,Alle Wohnungen,1,Wohnung,1312.0,6782,2348,253


In [14]:
# save current DF to csv just for development (for use in spacial data notebook)
df.to_csv('rentalprices_temp.csv', sep=';')